In [9]:
import psycopg2
import psycopg2.extras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import figure
import dataframe_image as dfi

In [10]:
connection = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="password",
    host="127.0.0.1",
    port="5432"
)
db = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)


In [11]:
def getAll(query):
    db.execute(query)
    return pd.DataFrame(data=db.fetchall())

In [12]:
# top 10 userów którzy napisali najwięcej tweetów

sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba tweetów"
from "user" u
         join tweet t on u.id = t.user_id
where t.reply_to is null
  and t.type = 1
group by u.id, u.name
order by count(*) desc
limit 10;
"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-tweetow.png")
df1


In [14]:
# top 10 userów którzy napisali najwięcej komentarzy

sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba komentarzy"
from "user" u
         join tweet t on u.id = t.user_id
where t.type = 2
group by u.id, u.name
order by count(*) desc
limit 10;
"""

df1 = getAll(sql1)
# dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-komentarzy.png")
df1

,Nazwa użytkownika,Liczba komentarzy
0,GetVideoBot,89
1,🌺 KikaGL 🌺,84
2,GetVideoBot 🤖,76
3,Camel,67
4,DownloadThisVideo🤖,64
5,Thread Reader App,62
6,Eleonora Cole,53
7,Neena Jha 💙,46
8,Eric Feigl-Ding,42
9,Tatiana Schild,40


In [ ]:
# top 10 userów którzy napisali najwięcej pozytywnych tweetow


sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba pozytywnych tweetow"
from "user" u
         join tweet t on u.id = t.user_id
where t.reply_to is null
  and t.type = 1
  and t.sentiment_compound >= 0.05
group by u.id, u.name
order by count(*) desc
limit 10;
"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-pozytywnych-tweetow.png")
df1

,Nazwa użytkownika,Liczba pozytywnych tweetow
0,Henry Niman,359
1,david pietrzak,272
2,P&O Ferries Freight,214
3,GlobalPandemic.NET,206
4,SpeechTrans,199
5,The Hill,198
6,PulpNews Crime,195
7,CNN,129
8,Reuters,110
9,ExBulletin,108


In [ ]:
# top 10 userów którzy napisali najwięcej negatywnych tweetow


sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba negatywnych tweetow"
from "user" u
         join tweet t on u.id = t.user_id
where t.reply_to is null
  and t.type = 1
  and t.sentiment_compound <= -0.05
group by u.id, u.name
order by count(*) desc
limit 10;

"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-negatywnych-tweetow.png")
df1

,Nazwa użytkownika,Liczba negatywnych tweetow
0,Covid Counting,195
1,COVIDBot,174
2,The Hill,153
3,Ray Giles,140
4,PulpNews Crime,140
5,COVID-19 Brasil Updates,136
6,CNN,131
7,Eric Feigl-Ding,128
8,Reuters,127
9,Tomthunkit™,123


In [ ]:
# top 10 userów którzy napisali najwięcej pozytywnych komentarzy


sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba pozytywnych komentarzy"
from "user" u
         join tweet t on u.id = t.user_id
where t.type = 2
  and t.sentiment_compound >= 0.05
group by u.id, u.name
order by count(*) desc
limit 10;

"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-pozytywnych-komentarzy.png")
df1

,Nazwa użytkownika,Liczba pozytywnych komentarzy
0,GetVideoBot,76
1,GetVideoBot 🤖,45
2,Thread Reader App,45
3,Neena Jha 💙,27
4,DownloadThisVideo🤖,24
5,Terry,24
6,Garrett Popcorn,22
7,Nicholas A. Christakis,20
8,Glasgow Women's Library,18
9,Def Noodles,18


In [ ]:
# top 10 userów którzy napisali najwięcej negatywnych komentarzy

sql1 = """
select u.name as "Nazwa użytkownika", count(*) as "Liczba negatywnych komentarzy"
from "user" u
         join tweet t on u.id = t.user_id
where t.type = 2
  and t.sentiment_compound <= -0.05
group by u.id, u.name
order by count(*) desc
limit 10;

"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-ktorzy-napisali-najwiecej-negatywnych-komentarzy.png")
df1

,Nazwa użytkownika,Liczba negatywnych komentarzy
0,La sagrada,19
1,Eric Feigl-Ding,16
2,Jeremy Konyndyk,15
3,Dr. Angela Rasmussen,15
4,i. 🧈,14
5,Megan Ranney MD MPH 🗽,14
6,Old Spice LA,14
7,Muge Cevik,13
8,IM,12
9,Lapsyrévoltée,12


In [ ]:
# top 10 najpopularniejszych tweetow z największą ilością komentarzy + retweetów + cytowań + polubień

sql1 = """
select t.id as "Identyfikator tweeta", t.content as "Treść tweeta", t.reply_count + t.retweet_count + t.quote_count + t.favorite_count as "Suma komentarzy, retweetów cytowań i polubień"
from tweet t
where t.type = 1
order by "Suma komentarzy, retweetów cytowań i polubień" desc
limit 10;
"""

df1 = getAll(sql1)
dfi.export(df1,"images/top-10-userow-najpopularniejszych-tweetow.png")
df1

,Identyfikator tweeta,Treść tweeta,"Suma komentarzy, retweetów cytowań i polubień"
0,1351897267666608129,It’s a new day in America.,4860898
1,1266821069823975430,Congratulations to the Astronauts that left Ea...,4457232
2,1265832328598884352,What a headline https://t.co/x8GGKL0VMc,3628671
3,1278763679421431809,You can have an edit button when everyone wear...,3575193
4,1268422690336935943,"우리는 인종차별에 반대합니다.\n우리는 폭력에 반대합니다.\n나, 당신, 우리 모두...",3318262
5,1383464956859879426,그립당 그때 그 시절 ㅠ..ㅠ💜 https://t.co/mVsupCoXJo,3133256
6,1331363238487977985,I can’t even believe these pics are real and i...,2224281
7,1266484584683896832,Racism isn’t a touchy topic if you’re not a fu...,2217700
8,1311120831032909825,Donald Trump is the worst president we've ever...,2208124
9,1266156916121706502,"It’s not white vs black, it’s everybody vs rac...",2043345
